<a href="https://colab.research.google.com/github/DanhChun/Log-Analyzer-with-NLP-model/blob/main/Logs_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Đọc dữ liệu từ file CSV
from google.colab import files
uploaded = files.upload()

Saving labeled_logs.csv to labeled_logs (2).csv


In [ ]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder

# Đọc file CSV
df = pd.read_csv("labeled_logs.csv")

# Chuẩn hóa cột "labels" (loại bỏ khoảng trắng thừa)
df["labels"] = df["labels"].str.strip()

# Kiểm tra nhãn trong dữ liệu
unique_labels = df["labels"].unique()
print("Các nhãn có trong dữ liệu:", unique_labels)

# Tìm và sửa lỗi nhãn không đúng
for label in unique_labels:
    if "NormalERROR" in label or "ERRORNormal" in label:
        print("Nhãn có vấn đề:", label)

# Sửa lỗi nhãn bằng cách thay thế tất cả trường hợp lỗi
df["labels"] = df["labels"].replace({
    "NormalERROR - SIM, GSM": "Normal"
})

# Mã hóa nhãn bằng LabelEncoder
label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["labels"])

# Kiểm tra ánh xạ sau khi sửa
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Mapping giữa nhãn và số sau khi sửa:", label_mapping)

# Lưu LabelEncoder để sử dụng sau này
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

# Xuất file CSV đã xử lý
df.to_csv("processed_logs.csv", index=False)

print("Dữ liệu đã được xử lý và lưu thành công!")


Các nhãn có trong dữ liệu: ['Normal' 'NOTE - all threads closed'
 'NOTE - speaker changes state from On to Off'
 'NOTE - speaker changes state from Off to On' 'ERROR - control amplifier'
 'WARNING - On disconnect' 'ERROR - start MPD failed'
 'ERROR - delete message' 'ERROR - MPD problem' 'ERROR - config SIM, GSM'
 'ERROR - invalid checksum' 'ERROR - invalid message size'
 'Note - message played done' 'ERROR - message download']
Mapping giữa nhãn và số sau khi sửa: {'ERROR - MPD problem': np.int64(0), 'ERROR - config SIM, GSM': np.int64(1), 'ERROR - control amplifier': np.int64(2), 'ERROR - delete message': np.int64(3), 'ERROR - invalid checksum': np.int64(4), 'ERROR - invalid message size': np.int64(5), 'ERROR - message download': np.int64(6), 'ERROR - start MPD failed': np.int64(7), 'NOTE - all threads closed': np.int64(8), 'NOTE - speaker changes state from Off to On': np.int64(9), 'NOTE - speaker changes state from On to Off': np.int64(10), 'Normal': np.int64(11), 'Note - message pl

In [ ]:
# Sửa các nhãn có vấn đề
df["labels"] = df["labels"].replace({"NormalERROR - SIM, GSM": "Normal"})  # Hoặc sửa theo đúng nhãn mong muốn

# Đảm bảo tất cả nhãn đều hợp lệ
df["labels"] = df["labels"].apply(lambda x: x.strip() if isinstance(x, str) else x)

# Kiểm tra lại
print("Danh sách nhãn sau khi làm sạch:", df["labels"].unique())


Danh sách nhãn sau khi làm sạch: ['Normal' 'NOTE - all threads closed'
 'NOTE - speaker changes state from On to Off'
 'NOTE - speaker changes state from Off to On' 'ERROR - control amplifier'
 'WARNING - On disconnect' 'ERROR - start MPD failed'
 'ERROR - delete message' 'ERROR - MPD problem' 'ERROR - config SIM, GSM'
 'ERROR - invalid checksum' 'ERROR - invalid message size'
 'Note - message played done' 'ERROR - message download']


**TOKENIZE DỮ LIỆU VÀ CHIA DỮ LIỆU**

In [ ]:
# Cài đặt thư viện cần thiết
!pip install torch transformers datasets scikit-learn pandas

import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import DistilBertTokenizer

# Đọc file CSV
df = pd.read_csv("processed_logs.csv", encoding="utf-8")

# Kiểm tra các cột có trong DataFrame
print("Tên các cột trong DataFrame:", df.columns.tolist())

# Kiểm tra nếu 'logs' không tồn tại thì báo lỗi
if "logs" not in df.columns:
    raise ValueError("Cột 'logs' không tồn tại trong file CSV! Kiểm tra lại tên cột.")

# Loại bỏ các dòng có giá trị NaN trong logs
df = df.dropna(subset=["logs"])

# Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
max_length = 128  # Độ dài tối đa của chuỗi token

tokens = tokenizer(
    df["logs"].tolist(),  # Chuyển logs thành danh sách trước khi token hóa
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

# Dataset PyTorch
class LogDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Kiểm tra xem 'label_encoded' có tồn tại không
if "label_encoded" not in df.columns:
    raise ValueError("Cột 'label_encoded' không tồn tại! Kiểm tra lại dữ liệu.")

# Chuyển nhãn thành tensor
labels = torch.tensor(df["label_encoded"].values, dtype=torch.long)

# Tạo dataset
dataset = LogDataset(tokens, labels)

# Chia tập train/test (80/20)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Tạo DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) # Create val_loader


# Kiểm tra số lượng mẫu
print(f"Tổng số mẫu: {len(dataset)}")
print(f"Số mẫu train: {len(train_dataset)}")
print(f"Số mẫu test: {len(test_dataset)}")


Tên các cột trong DataFrame: ['logs', 'labels', 'label_encoded']
Tổng số mẫu: 4820
Số mẫu train: 3856
Số mẫu test: 482


In [ ]:
!pip install torch transformers datasets scikit-learn


**CÀI ĐẶT CUDA**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import DistilBertForSequenceClassification

# Kiểm tra GPU (nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Khởi tạo mô hình với DistilBERT
num_labels = len(df["label_encoded"].unique())  # Số nhãn đầu ra
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)
model.to(device)

# Định nghĩa hàm mất mát và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)


Sử dụng thiết bị: cuda


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_masks[idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)  # Dòng này gây cảnh báo
        }

        # Kiểm tra nếu labels chưa là tensor, thì chuyển đổi
        label = self.labels[idx]
        if not isinstance(label, torch.Tensor):
            label = torch.tensor(label, dtype=torch.long)

        item["labels"] = label  # Gán nhãn vào item
        return item

    def __len__(self):
        return len(self.labels)


**CẤU HÌNH TRAIN VÀ TIẾN HÀNH TRAIN**

In [ ]:
# Kiểm tra train_loader có tồn tại không
assert 'train_loader' in globals(), "train_loader chưa được khởi tạo!"

# Kiểm tra batch có đủ input_ids, attention_mask, labels
sample_batch = next(iter(train_loader))
assert "input_ids" in sample_batch and "attention_mask" in sample_batch and "labels" in sample_batch, \
    "Batch không chứa đủ input_ids, attention_mask, labels!"


<ipython-input-25-dba6642a4671>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)


In [ ]:
from tqdm import tqdm  # Hiển thị tiến trình huấn luyện

# Hàm tính độ chính xác
def compute_accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            # Dự đoán nhãn
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(data_loader)
    accuracy = correct / total
    return avg_loss, accuracy

# Số epoch huấn luyện
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)
    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    # Tính loss & accuracy trên tập train
    train_loss = total_loss / len(train_loader)

    # Tính loss & accuracy trên tập validation
    val_loss, val_accuracy = compute_accuracy(model, val_loader)

    print(f"\nEpoch {epoch+1}/{epochs}:")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4%}")
    print("-" * 50)


Epoch 1/3:   0%|          | 0/241 [00:00<?, ?it/s]<ipython-input-25-dba6642a4671>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
Epoch 1/3:   5%|▍         | 11/241 [00:01<00:39,  5.87it/s, loss=1.21]<ipython-input-25-dba6642a4671>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
Epoch 1/3:  17%|█▋        | 40/241 [00:06<00:36,  5.58it/s, loss=1.43]<ipython-input-25-dba6642a4671>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.


Epoch 1/3:
Train Loss: 0.4268
Val Loss: 0.0220, Val Accuracy: 99.5851%
--------------------------------------------------


Epoch 2/3:   2%|▏         | 5/241 [00:00<00:42,  5.62it/s, loss=0.00921]<ipython-input-25-dba6642a4671>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
Epoch 2/3:   7%|▋         | 18/241 [00:03<00:38,  5.76it/s, loss=0.00854]<ipython-input-25-dba6642a4671>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
Epoch 2/3:  14%|█▍        | 34/241 [00:05<00:36,  5.70it/s, loss=0.00801]<ipython-input-25-dba6642a4671>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_gra


Epoch 2/3:
Train Loss: 0.0253
Val Loss: 0.0087, Val Accuracy: 99.5851%
--------------------------------------------------


Epoch 3/3:   7%|▋         | 17/241 [00:02<00:38,  5.83it/s, loss=0.00333]<ipython-input-25-dba6642a4671>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
Epoch 3/3:  13%|█▎        | 31/241 [00:05<00:36,  5.78it/s, loss=0.00308]<ipython-input-25-dba6642a4671>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
Epoch 3/3:  20%|██        | 49/241 [00:08<00:32,  5.85it/s, loss=0.00146]<ipython-input-25-dba6642a4671>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_gr


Epoch 3/3:
Train Loss: 0.0142
Val Loss: 0.0030, Val Accuracy: 100.0000%
--------------------------------------------------


In [ ]:
import os

# Tạo thư mục lưu mô hình
model_dir = "distilbert_log_classifier"
os.makedirs(model_dir, exist_ok=True)

# Lưu mô hình
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

print(f" Mô hình và tokenizer đã được lưu vào thư mục: {model_dir}")


 Mô hình và tokenizer đã được lưu vào thư mục: distilbert_log_classifier


In [ ]:
import pickle

# Lưu LabelEncoder
label_encoder_path = os.path.join(model_dir, "label_encoder.pkl")
with open(label_encoder_path, "wb") as f:
    pickle.dump(label_encoder, f)

print(f" LabelEncoder đã được lưu tại: {label_encoder_path}")


 LabelEncoder đã được lưu tại: distilbert_log_classifier/label_encoder.pkl


In [ ]:
import os
print(" Danh sách tệp đã lưu:")
print(os.listdir(model_dir))


 Danh sách tệp đã lưu:
['model.safetensors', 'vocab.txt', 'tokenizer_config.json', 'special_tokens_map.json', 'config.json', 'label_encoder.pkl']


**TẢI LẠI MÔ HÌNH VÀ TOKENIZER**

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import pickle

# Định nghĩa đường dẫn mô hình đã lưu
model_dir = "distilbert_log_classifier"

# Tải mô hình & tokenizer
model = DistilBertForSequenceClassification.from_pretrained(model_dir)
tokenizer = DistilBertTokenizer.from_pretrained(model_dir)

# Đưa mô hình lên GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print(" Mô hình và tokenizer đã được tải thành công!")


 Mô hình và tokenizer đã được tải thành công!


**TẢI LẠI LABEL ENCODER**

In [ ]:
# Tải LabelEncoder để giải mã nhãn
label_encoder_path = f"{model_dir}/label_encoder.pkl"
with open(label_encoder_path, "rb") as f:
    label_encoder = pickle.load(f)

print(" LabelEncoder đã được tải thành công!")


 LabelEncoder đã được tải thành công!


**KIỂM TRA MÔ HÌNH VỚI 1 LOG**

In [ ]:
log_message = "System overheating detected. Close all threads in 5 seconds."
# Tokenize dữ liệu
inputs = tokenizer(log_message, return_tensors="pt", padding=True, truncation=True)
inputs = {key: val.to(device) for key, val in inputs.items()}  # Đưa lên GPU nếu có

# Thực hiện suy luận
with torch.no_grad():
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).cpu().item()

# Giải mã nhãn từ số thành nội dung cảnh báo
decoded_label = label_encoder.inverse_transform([predicted_label])[0]

print(f" Log: {log_message}")
print(f"Cảnh báo dự đoán: {decoded_label}")


 Log: System overheating detected. Close all threads in 5 seconds.
Cảnh báo dự đoán: Normal


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="fine_tuned_model",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

trainer.save_model("fine_tunned_model")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

**KIỂM TRA MÔ HÌNH VỚI NHIỀU LOG**

In [ ]:
log_samples = [
    "Temperature sensor failure detected.",
    "Unauthorized access attempt detected.",
    "Battery voltage critically low.",
    "Network connection lost. Reconnecting..."
]

for log in log_samples:
    inputs = tokenizer(log, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Đưa lên GPU nếu có

    with torch.no_grad():
        outputs = model(**inputs)
        predicted_label = torch.argmax(outputs.logits, dim=1).cpu().item()

    decoded_label = label_encoder.inverse_transform([predicted_label])[0]

    print(f" Log: {log}")
    print(f" Cảnh báo dự đoán: {decoded_label}")
    print("-" * 50)


 Log: Temperature sensor failure detected.
 Cảnh báo dự đoán: Normal
--------------------------------------------------
 Log: Unauthorized access attempt detected.
 Cảnh báo dự đoán: Normal
--------------------------------------------------
 Log: Battery voltage critically low.
 Cảnh báo dự đoán: Normal
--------------------------------------------------
 Log: Network connection lost. Reconnecting...
 Cảnh báo dự đoán: Normal
--------------------------------------------------


**LƯU LẠI MÔ HÌNH TRAIN**

In [ ]:
from transformers import TFDistilBertForSequenceClassification

# Tải lại mô hình từ thư mục đã lưu trước đó
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert_log_classifier", from_pt=True)

# Chuyển mô hình sang định dạng SavedModel (để có saved_model.pb)
export_dir = "saved_model_distilbert"
model.save_pretrained(export_dir, saved_model=True)

print(f" Mô hình đã được chuyển sang định dạng TensorFlow và lưu tại: {export_dir}")


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


 Mô hình đã được chuyển sang định dạng TensorFlow và lưu tại: saved_model_distilbert


**TỐI ƯU MÔ HÌNH**

**Convert to TF lite**

In [ ]:
!pip install torch transformers datasets scikit-learn
!pip install torch-tflite

import torch
import tensorflow as tf
from transformers import DistilBertForSequenceClassification

# Load mô hình đã quantized
model_int8 = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels = 15)
model_int8.load_state_dict(torch.load("distilbert_qat_int8.pth"))
model_int8.eval()

# Chuyển đổi sang TorchScript
scripted_model = torch.jit.trace(lambda x, y: model_int8(x, attention_mask = y).logits, (torch.randint(0, 30522, (1, 128)), torch.ones((1, 128), dtype=torch.long)))
scripted_model.save("distilbert_qat_int8.pt")

# Chuyển sang TFLite
import torch_tflite
tflite_model = torch_tflite.convert(scripted_model)
open("distilbert_qat_int8.tflite", "wb").write(tflite_model)


ERROR: Could not find a version that satisfies the requirement torch-tflite (from versions: none)
ERROR: No matching distribution found for torch-tflite


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: [Errno 2] No such file or directory: 'distilbert_qat_int8.pth'

**CHUYỂN MÔ HÌNH SANG TF**

In [ ]:
import os

# Kiểm tra thư mục đã lưu mô hình
export_dir = "saved_model_distilbert/saved_model"

if os.path.exists(export_dir):
    print(" Mô hình đã lưu thành công!")
    print(f" Danh sách file trong {export_dir}: {os.listdir(export_dir)}")
else:
    print(" Lưu mô hình thất bại! Hãy kiểm tra lại quá trình lưu mô hình.")


In [ ]:
import tensorflow as tf

# Load mô hình đã huấn luyện
model = tf.keras.models.load_model("saved_model_distilbert/saved_model/1/")

# Chuyển đổi sang TFLite
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model_distilbert/saved_model/1/")
tflite_model = converter.convert()

# Lưu file .tflite
with open("distilbert_log_classifier.tflite", "wb") as f:
    f.write(tflite_model)

print(" Chuyển đổi thành công! File đã lưu.")


In [ ]:
print(input_details)


In [ ]:
import tensorflow.lite as tflite
import numpy as np
from transformers import DistilBertTokenizer

# Load tokenizer giống như lúc train
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Load model
interpreter = tflite.Interpreter(model_path="distilbert_log_classifier.tflite")
interpreter.allocate_tensors()

# Lấy thông tin tensor input/output
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Chuyển đổi câu log thành tensor (chỉ lấy token đầu tiên)
test_log = "Close all threads"
tokens = tokenizer.encode(test_log, truncation=True)

# Chỉ lấy 1 token đầu tiên để phù hợp shape [1, 1]
first_token = tokens[0] if len(tokens) > 0 else 0

# Đảm bảo đúng shape [1, 1]
input_ids = np.array([[first_token]], dtype=np.int64)
attention_mask = np.array([[1]], dtype=np.int64)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], attention_mask)  # attention mask
interpreter.set_tensor(input_details[1]['index'], input_ids)  # input IDs

# Chạy mô hình
interpreter.invoke()

# Lấy kết quả
output = interpreter.get_tensor(output_details[0]['index'])
predicted_label = np.argmax(output)

print(f"Log: {test_log}")
print(f"Predicted Label: {predicted_label}")


**XỬ LÝ ĐẦU VÀO MÔ HÌNH CHO QUÁ TRÌNH PTQ**

In [ ]:
import tensorflow as tf

# Load lại mô hình từ SavedModel
model = tf.keras.models.load_model("saved_model_distilbert/saved_model/1", compile=False)

# Chuyển mô hình về dạng `tf.function` để cố định input shape
@tf.function(input_signature=[
    tf.TensorSpec(shape=[1, 128], dtype=tf.int32, name="input_ids"),
    tf.TensorSpec(shape=[1, 128], dtype=tf.int32, name="attention_mask"),
])
def serving_fn(input_ids, attention_mask):
    outputs = model({"input_ids": input_ids, "attention_mask": attention_mask})
    logits = outputs["logits"]  # Truy cập logits từ dictionary
    return {"logits": tf.identity(logits, name="logits")}  # Đảm bảo output là tf.Tensor

# Lưu mô hình mới với input cố định
fixed_model_dir = "saved_model_distilbert_fixed"
tf.saved_model.save(model, fixed_model_dir, signatures={"serving_default": serving_fn})

print(f"Mô hình đã lưu với input cố định tại {fixed_model_dir}")


**THỰC HIỆN PTQ INT8**

In [ ]:
import tensorflow as tf
import numpy as np

# Load mô hình đã sửa
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model_distilbert_fixed")

# Bật lượng tử hóa trọng số (INT8)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Sử dụng tập dữ liệu đại diện để giúp lượng tử hóa
def representative_dataset_gen():
    for _ in range(100):
        input_ids = np.random.randint(0, 30522, size=(1, 128), dtype=np.int32)
        attention_mask = np.ones((1, 128), dtype=np.int32)
        yield [input_ids, attention_mask]  # Trả về tuple thay vì dictionary

converter.representative_dataset = representative_dataset_gen

# Cấu hình kiểu dữ liệu đầu vào và đầu ra của mô hình
converter.inference_input_type = tf.int8   # Đầu vào INT8
converter.inference_output_type = tf.int8  # Đầu ra INT8

# Chuyển đổi mô hình sang TFLite
tflite_model = converter.convert()

# Lưu mô hình mới
with open("distilbert_log_classifier_int8.tflite", "wb") as f:
    f.write(tflite_model)

print(" Chuyển đổi thành công!")


In [ ]:
import tensorflow as tf
import numpy as np

# Load mô hình TFLite INT8
interpreter = tf.lite.Interpreter(model_path="distilbert_log_classifier_int8.tflite")
interpreter.allocate_tensors()

# Lấy thông tin tensor đầu vào & đầu ra
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Thông tin đầu vào:", input_details)
print("Thông tin đầu ra:", output_details)

# Tạo dữ liệu test (để nguyên dtype int32)
input_ids = np.random.randint(0, 30522, size=(1, 128), dtype=np.int32)
attention_mask = np.ones((1, 128), dtype=np.int32)

# Đưa dữ liệu vào mô hình (KHÔNG CHUYỂN SANG int8)
interpreter.set_tensor(input_details[0]["index"], attention_mask)
interpreter.set_tensor(input_details[1]["index"], input_ids)

# Chạy mô hình
interpreter.invoke()

# Lấy kết quả đầu ra & giải lượng tử hóa
output_scale, output_zero_point = output_details[0]['quantization']
output_data_q = interpreter.get_tensor(output_details[0]["index"])
output_data = (output_data_q.astype(np.float32) - output_zero_point) * output_scale

print("Kết quả đầu ra (logits float32):", output_data)


In [ ]:
predicted_label = np.argmax(output_data, axis=-1)
print("Nhãn dự đoán:", predicted_label)


In [ ]:
import os

def get_dir_size(path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size

export_dir = "saved_model_distilbert/saved_model"

size_mb = get_dir_size(export_dir) / (1024 * 1024)

print(f" Kích thước mô hình gốc: {size_mb:.2f} MB")
print(f" Kích thước INT8: {os.path.getsize('distilbert_log_classifier_int8.tflite') / (1024 * 1024):.2f} MB")


In [ ]:
# Load mô hình TFLite đã lượng tử hóa
interpreter_int8 = tf.lite.Interpreter(model_path="distilbert_log_classifier_int8.tflite")

# Kiểm tra thông tin mô hình
print("INT8 Model Inputs:", interpreter_int8.get_input_details())


In [ ]:
print(os.listdir("saved_model_distilbert/saved_model"))


In [ ]:
import os

print("Tệp INT8 tồn tại:", os.path.exists("distilbert_log_classifier_int8.tflite"))


In [ ]:
import time
import numpy as np
import tensorflow.lite as tflite

# Load mô hình INT8
interpreter_int8 = tflite.Interpreter(model_path="distilbert_log_classifier_int8.tflite")
interpreter_int8.allocate_tensors()


# Chuẩn bị dữ liệu test (có thể lấy từ logs thực tế)
test_input = {
    "attention_mask": np.ones((1, 128), dtype=np.int32),
    "input_ids": np.random.randint(0, 30522, size=(1, 128), dtype=np.int32)
}

def run_inference(interpreter, test_input):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    for i, key in enumerate(test_input.keys()):
        interpreter.set_tensor(input_details[i]['index'], test_input[key])

    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data, end_time - start_time

# Chạy inference và đo thời gian
output_int8, time_int8 = run_inference(interpreter_int8, test_input)

print(f"Output INT8: {output_int8}, Time: {time_int8:.4f}s")


In [ ]:
threshold = 50  # Với INT8, giá trị nằm trong khoảng [-128, 127] nên chọn ngưỡng cao hơn
labels_int8 = [log_labels_example[i] for i in range(len(output_int8[0])) if output_int8[0][i] > threshold]



print(f" Cảnh báo INT8: {', '.join(labels_int8) if labels_int8 else 'Không có lỗi'}")


In [ ]:
import pandas as pd

logs_df = pd.read_csv("/content/labeled_logs.csv")
print("Các cột trong CSV:", logs_df.columns)


In [ ]:
print(logs_df["logs"].head())
print(logs_df["logs"].dtype)  # Kiểm tra kiểu dữ liệu của cột logs


In [ ]:
import numpy as np
import pandas as pd
import tensorflow.lite as tflite
from transformers import DistilBertTokenizer

# ======== BƯỚC 1: Load file logs.csv ========
log_file = "/content/labeled_logs.csv"
try:
    logs_df = pd.read_csv(log_file)
    assert "logs" in logs_df.columns, " Cột 'logs' không tồn tại trong file CSV!"

    # Chuyển tất cả giá trị về chuỗi, thay NaN bằng chuỗi rỗng
    logs_df["logs"] = logs_df["logs"].fillna("").astype(str)
except Exception as e:
    raise SystemExit(f" Lỗi khi đọc file CSV: {e}")

# ======== BƯỚC 2: Tokenizer thật ========
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_log(log_message):
    try:
        tokens = tokenizer(
            log_message, padding="max_length", truncation=True, max_length=128, return_tensors="np"
        )
        return {
            "input_ids": tokens["input_ids"].astype(np.int32),
            "attention_mask": tokens["attention_mask"].astype(np.int32)
        }
    except Exception as e:
        print(f"Lỗi khi tokenize log: {log_message} → {e}")
        return {
            "input_ids": np.zeros((1, 128), dtype=np.int32),
            "attention_mask": np.zeros((1, 128), dtype=np.int32)
        }

logs_data = [tokenize_log(msg) for msg in logs_df["logs"]]

# ======== BƯỚC 3: Load mô hình INT8 ========
try:
    interpreter_int8 = tflite.Interpreter(model_path="distilbert_log_classifier_int8.tflite")


    interpreter_int8.allocate_tensors()

except Exception as e:
    raise SystemExit(f" Lỗi khi load mô hình: {e}")

# ======== BƯỚC 4: Mapping số nhãn → nội dung cảnh báo ========
log_labels_mapping = {
    0: "ERROR - MPD problem", 1: "ERROR - control amplifier",
    2: "ERROR - delete message", 3: "ERROR - download message",
    4: "ERROR - invalid checksum", 5: "ERROR - invalid message size",
    6: "ERROR - message download", 7: "ERROR - start MPD failed",
    8: "ERROR: config SIM, GSM error", 9: "NOTE - all threads closed",
    10: "NOTE - speaker changes state from Off to On",
    11: "NOTE - speaker changes state from On to Off",
    12: "Normal",
    13: "Note - message played done", 14: "UNKNOWN LOG", 15: "WARNING - On disconnect"
}

# ======== BƯỚC 5: Chạy inference ========
def run_inference(interpreter, log_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], log_data["input_ids"])
    interpreter.set_tensor(input_details[1]['index'], log_data["attention_mask"])
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data

for i, log_data in enumerate(logs_data):
    output_int8 = run_inference(interpreter_int8, log_data)

    predicted_label_int8 = np.argmax(output_int8)

    # Tránh lỗi KeyError khi tra cứu nhãn
    label_int8 = log_labels_mapping.get(predicted_label_int8, "Unknown label")

    print(f"Log {i+1}: {logs_df['logs'][i]}")
    print(f"  - INT8 Prediction: {label_int8}")
    print("-" * 50)
